<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads_ELEPHAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat May 14 14:17:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install elephas
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 49.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a2ccca542965283fd6f89e38838d25cbf94f0a89ac3f56c78372da0f5fef24b8
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 4.1 MB 31.6 MB/s 
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 40.7 MB/s 
     |████████████████████████████████| 462 kB 61.5 MB/s 
  Created wheel for elephas: filename=elephas-3.1.0-py3-none-any.whl size=26259 sha256=bc88b2615f1c3ca3f6032212b2a9eb5700838c024485c2d51461e78ea6ec06a5
  Stored in directory: /root/.cache/pip/wheels/33/e4/e1/56dda8be927bb0e9971cd7ddf3fc1b17ce78db56268b1f867f
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=5399

In [1]:
import pyspark
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

# Keras / Deep Learning
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from tensorflow.keras.optimizers import Adam

# Elephas for Deep Learning on Spark
from elephas.ml_model import ElephasEstimator

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
from google.colab import files
import os
import zipfile
import findspark
from pyspark.sql.functions import col
from petastorm import TransformSpec
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import numpy as np

#from my model
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import pickle
import shutil
import random
import skimage.io as io
from copy import deepcopy



/usr/local/lib/python3.7/dist-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [4]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json


In [5]:
os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")


!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

 99% 2.15G/2.18G [00:18<00:00, 142MB/s]
100% 2.18G/2.18G [00:18<00:00, 128MB/s]


In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

findspark.init(findspark.find())
#findspark.init(findspark.find())
#spark = SparkSession.builder.master("local").appName("Colab").getOrCreate()
#spark

spark = SparkSession.builder.master("local[*]").appName("Colab").getOrCreate()
conf = SparkConf().setMaster("local[*]").setAppName("Colab")
conf.set("spark.executor.memory", "6G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")
sc =  SparkContext.getOrCreate(conf=conf)
spark

In [4]:
sql_context = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)

In [5]:
df_comics = spark.read.format("binaryFile").load(".//Face_Comics_data//comics").withColumn("label", lit(int(1)))
df_faces = spark.read.format("binaryFile").load(".//Face_Comics_data//faces").withColumn("label", lit(int(0)))

#df_comics.select("image.height","image.width","image.nChannels", "image.mode", "image.data").show(1)
#df_comics.show(5)
df_comics = df_comics.select(col("content"),col("label"))
df_faces = df_faces.select(col("content"),col("label"))

df_comics.show(5)
df_faces.show(5)

+--------------------+-----+
|             content|label|
+--------------------+-----+
|[FF D8 FF E0 00 1...|    1|
|[FF D8 FF E0 00 1...|    1|
|[FF D8 FF E0 00 1...|    1|
|[FF D8 FF E0 00 1...|    1|
|[FF D8 FF E0 00 1...|    1|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|             content|label|
+--------------------+-----+
|[FF D8 FF E0 00 1...|    0|
|[FF D8 FF E0 00 1...|    0|
|[FF D8 FF E0 00 1...|    0|
|[FF D8 FF E0 00 1...|    0|
|[FF D8 FF E0 00 1...|    0|
+--------------------+-----+
only showing top 5 rows



*****************************************!NB Setup Single node test*****************************************

In [4]:
conf = SparkConf().setAppName('Face_Comics_recognition').setMaster('local[6]')
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"


findspark.init(findspark.find())
spark = SparkSession.builder.master("local").appName("Colab").getOrCreate()
spark

In [ ]:
#limit files in order to test the algorithm before to distribute and scale

#os.makedirs("./a/sample_data")
#! ls -lt sample_data/*.csv | head -3 | awk '{print "mv " $9 " a/"$9}' | sh
#! rm -rf './sample_data'
#! mv './a/sample_data' './'
#! rm -rf './a'

os.makedirs("./a/Face_Comics_data/comics")
os.makedirs("./a/Face_Comics_data/faces")
! ls -lt Face_Comics_data/comics/*.jpg | head -10 | awk '{print "mv " $9 " a/"$9}' | sh
! ls -lt Face_Comics_data/faces/*.jpg | head -10 | awk '{print "mv " $9 " a/"$9}' | sh
! rm -rf './Face_Comics_data'
! mv './a/Face_Comics_data/' './'
! rm -rf './a'


In [6]:
df_comics = spark.read.format("binaryFile").load(".//Face_Comics_data//comics").withColumn("label", lit(int(1)))
df_faces = spark.read.format("binaryFile").load(".//Face_Comics_data//faces").withColumn("label", lit(int(0)))

#df_comics.select("image.height","image.width","image.nChannels", "image.mode", "image.data").show(1)
#df_comics.show(5)
df_comics = df_comics.select(col("content"),col("label"))
df_faces = df_faces.select(col("content"),col("label"))

#df_comics.show(5)
#df_faces.show(5)

In [6]:
df_comics = spark.read.format("image").load(".//Face_Comics_data//comics").withColumn("label", lit(int(1)))
df_faces = spark.read.format("image").load(".//Face_Comics_data//faces").withColumn("label", lit(int(0)))

#df_comics.select("image.height","image.width","image.nChannels", "image.mode", "image.data").show(1)
#df_comics.show(5)
df_comics = df_comics.select(col("image.data"),col("label"))
df_faces = df_faces.select(col("image.data"),col("label"))

#df_comics.show(5)
#df_faces.show(5)

In [6]:
#Read images and Create training & test DataFrames for transfer learning
#Shuffle Data
df_comics = df_comics.orderBy(rand())
df_faces = df_faces.orderBy(rand())

comics_train, comics_validation, comics_test = df_comics.randomSplit([0.6, 0.2,0.2], seed = 1234)
print(comics_train)
faces_train, faces_validation, faces_test = df_faces.randomSplit([0.6, 0.2,0.2], seed = 1234)
print(faces_train)

#dataframe for training a classification model
train_df = faces_train.unionAll(comics_train)

#dataframe for validating the classification model
val_df = faces_validation.unionAll(comics_validation)

#dataframe for testing the classification model
test_df = faces_test.unionAll(comics_test)

#num_classes = test_df.select("label").distinct().count()
train_df = train_df.repartition(2)
#print("number of classes", num_classes)

DataFrame[content: binary, label: int]
DataFrame[content: binary, label: int]


In [8]:
train_df

DataFrame[data: binary, label: int]

In [1]:
! rm -rf './staging'

In [18]:
from PIL import Image
import io

def preprocess(content):
  image = Image.open(io.BytesIO(content)).resize([224,224])
  image_array = keras.preprocessing.image.img_to_array(image)
  #return tf.keras.applications.mobilnte_v2.preprocess_input(image_array)
  return image_array

def transform_batch(pd_batch):
  pd_batch['features'] = pd_batch['content'].map(lambda x: preprocess(x))
  pd_batch = pd_batch.drop(labels='content', axis = 1)
  return pd_batch

IMG_SHAPE = (224, 224, 3)
EPOCHS = 15
BATCH_SIZE = 32
IMG_SIZE = (224, 224, 3)

transform_spec_fn = TransformSpec(
    transform_batch,
    edit_fields = [('features', np.float32, IMG_SHAPE, False)],
    selected_fields = ['features','label']
)

In [8]:
import pandas as pd
pd_batch0 = train_df.limit(5).toPandas()
pd_batch0

,content,label
0,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
2,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
3,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
4,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0


In [9]:
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, "file:/content/staging")

converter_train = make_spark_converter(train_df)
converter_val = make_spark_converter(val_df)

/usr/local/lib/python3.7/dist-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
Converting floating-point columns to float32
Converting floating-point columns to float32
The median size 5673339 B (< 50 MB) of the parquet files is too small. Total size: 463040157 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:///content/staging/20220515225541-appid-local-1652655118499-4ab63dbf-c0db-4b16-b9ad-e3897757e120/part-00057-5dd4226a-5a71-4dc8-b8c4-03e7cc57e278-c000.parquet, ...


In [ ]:
"""#pesca dalla folder scaricata, non dal flusso spark
data_dir= os.environ["data_source"]

os.listdir(data_dir)

tf.random.set_seed(123456)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    shuffle=True,
    seed=123456,
    image_size= (160,160),
    batch_size=32)


class_names = train_data.select("label_index").distinct().count()
print(class_names)
"""

In [19]:
with  converter_train.make_tf_dataset(transform_spec=transform_spec_fn, 
                                  batch_size = BATCH_SIZE) as train_dataset,\
        converter_val.make_tf_dataset(transform_spec=transform_spec_fn, 
                                  batch_size = BATCH_SIZE) as val_dataset:
                                    
    train_dataset_ = train_dataset.map(lambda x: (x.features, x.label))
    validation_dataset_ = val_dataset.map(lambda x: (x.features, x.label))

/usr/local/lib/python3.7/dist-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs


In [65]:
data_dir= "./Face_Comics_data"
os.listdir(data_dir)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    shuffle=True,
    seed=123456,
    image_size= IMG_SIZE,
    batch_size=BATCH_SIZE)


validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    shuffle=True,
    seed=123456,
    image_size= IMG_SIZE,
    batch_size=BATCH_SIZE)

Found 20000 files belonging to 2 classes.
Using 14000 files for training.
Found 20000 files belonging to 2 classes.
Using 6000 files for validation.


In [50]:
validation_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [15]:
validation_dataset_

<MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [26]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size=3, strides=2, activation='relu',  input_shape = [224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
cnn.add(tf.keras.layers.Dropout(0.1))
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model_fit = cnn.fit(x = train_dataset_, validation_data = validation_dataset_, epochs = EPOCHS)

cnn.summary()

Epoch 1/15


UnknownError: ignored

In [51]:
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, "file:/content/staging")

converter_train = make_spark_converter(train_df)
converter_val = make_spark_converter(val_df)

/usr/local/lib/python3.7/dist-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
The median size 5644755 B (< 50 MB) of the parquet files is too small. Total size: 462418027 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:///content/staging/20220515191308-appid-local-1652641712749-e4920d43-7e43-472e-836f-e10763ca4330/part-00069-c04df76c-2fb0-413f-8bd9-b66786401763-c000.parquet, ...


In [52]:
converter_val

In [43]:
from elephas.utils.rdd_utils import to_simple_rdd
from elephas.spark_model import SparkModel
from tensorflow import keras
import tensorflow as tf 

optimizer_conf = tf.keras.optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

# Initialize Elephas Spark ML Estimator
estimator = ElephasEstimator()
estimator.setFeaturesCol("content")
estimator.setLabelCol("label")
estimator.set_keras_model_config(cnn.to_json())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(num_classes)
estimator.set_num_workers(1)
estimator.set_epochs(15)
estimator.set_batch_size(32)
estimator.set_verbosity(1)
estimator.set_validation_split(0.20)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])


# Fitting a model returns a Transformer
pipeline = Pipeline(stages=[estimator])

"""
fitted_pipeline = pipeline.fit(train_df)

# Evaluate Spark model
prediction = fitted_pipeline.transform(train_df)
pnl = prediction.select("index_category", "prediction")
pnl.show
"""

"""
fit_dl_pipeline = pipeline.fit(train_data)
pred_train = fit_dl_pipeline.transform(train_data)
pred_test = fit_dl_pipeline.transform(test_data)

pnl_train = pred_train.select(label, "prediction")
pnl_test = pred_test.select(label, "prediction")

pred_and_label_train = pnl_train.rdd.map(lambda row: (row[label], row['prediction']))
pred_and_label_test = pnl_test.rdd.map(lambda row: (row[label], row['prediction']))

metrics_train = MulticlassMetrics(pred_and_label_train)
metrics_test = MulticlassMetrics(pred_and_label_tes

print("Training data accuracy: {}".format(round(metrics_train.precision(),4)))
print("Training Data Confusion Matrix")
display(pnl_train.crosstab('label','prediction').toPandas())

print("\nTest data accuracy: {}".format(round(metrics_test.precision(),4)))
print("Test Data Confusion Matrix")
display(pnl_test.crosstab('label','prediction').toPandas())
"""

#hist = cnn.fit(train_dataset, validation_data = val_dataset, epochs = EPOCHS)
#return hist.history['val_loss'][-1], hist.history['val_accuracy'][-1]
#return 

#loss, accuracy = training_and_validation()
#print('Validation accuracy: {}'.format(accuracy))


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


'\nfit_dl_pipeline = pipeline.fit(train_data)\npred_train = fit_dl_pipeline.transform(train_data)\npred_test = fit_dl_pipeline.transform(test_data)\n\npnl_train = pred_train.select(label, "prediction")\npnl_test = pred_test.select(label, "prediction")\n\npred_and_label_train = pnl_train.rdd.map(lambda row: (row[label], row[\'prediction\']))\npred_and_label_test = pnl_test.rdd.map(lambda row: (row[label], row[\'prediction\']))\n\nmetrics_train = MulticlassMetrics(pred_and_label_train)\nmetrics_test = MulticlassMetrics(pred_and_label_tes\n\nprint("Training data accuracy: {}".format(round(metrics_train.precision(),4)))\nprint("Training Data Confusion Matrix")\ndisplay(pnl_train.crosstab(\'label\',\'prediction\').toPandas())\n\nprint("\nTest data accuracy: {}".format(round(metrics_test.precision(),4)))\nprint("Test Data Confusion Matrix")\ndisplay(pnl_test.crosstab(\'label\',\'prediction\').toPandas())\n'

In [44]:
def dl_pipeline_fit_score_results(dl_pipeline=pipeline,
                                  train_data = train_dataset,
                                  test_data = val_dataset,
                                  label = 'label'):
  
    fit_dl_pipeline = dl_pipeline.fit(train_data)
    pred_train = fit_dl_pipeline.transform(train_data)
    pred_test = fit_dl_pipeline.transform(test_data)

    pnl_train = pred_train.select(label, "prediction")
    pnl_test = pred_test.select(label, "prediction")

    pred_and_label_train = pnl_train.rdd.map(lambda row: (row[label], row['prediction']))
    pred_and_label_test = pnl_test.rdd.map(lambda row: (row[label], row['prediction']))

    metrics_train = MulticlassMetrics(pred_and_label_train)
    metrics_test = MulticlassMetrics(pred_and_label_test)

    print("Training data accuracy: {}".format(round(metrics_train.precision(),4)))
    print("Training Data Confusion Matrix")
    display(pnl_train.crosstab('label','prediction').toPandas())

    print("\nTest data accuracy: {}".format(round(metrics_test.precision(),4)))
    print("Test Data Confusion Matrix")
    display(pnl_test.crosstab('label','prediction').toPandas())

In [45]:
dl_pipeline_fit_score_results(dl_pipeline=pipeline,
                                  train_data = train_dataset,
                                  test_data = val_dataset,
                                  label = 'label')

AttributeError: ignored

In [ ]:
#There are two variable objects. Divided between around 2.5 million of MobilNet parameters which 
#are frozen, and 1.2 thousend of trainable parameter in the Dense layer
len(cnn.trainable_variables)

acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
loss_ = model_fit.history['loss']
val_loss_ = model_fit.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color = 'gray',linestyle='dashed')
plt.plot(val_acc, label='Validation Accuracy', color = 'black')
plt.ylim([0.8, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss_, label='Training Loss', color = 'gray',linestyle='dashed')
plt.plot(val_loss_, label='Validation Loss', color = 'black')
plt.ylim([0, 1.0])
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
